# Diet Problem

# Problem statement

## Set

$F$ = set of foods

$N$ = set of nutrients

## Decision Variables

$x_{f}$ = Number of servings of food $f$

## Parameters

$c_{f}$ = Cost per serving of food $f$,  $\forall f \in F$

$a_{f,n}$ = Amount of nutrients $n$ in food $f$, $\forall f \in F, \forall n \in N$

$Nmin_{n}$ = Minimum level of nutrients $n$, $\forall n \in N$

$Nmax_{n}$ = Maximun level of nutrients $n$, $\forall n \in N$

$Vmax$ = Maximum volume of food comsumed

$V_{f}$ = The volume per serving of food $f$, $\forall f \in F



## Objective

Minimize the total cost of the food

$$\text{min} \sum c_f x_f$$

## Constrains

1. Limit nutrient consumption for each nutrient

$$Nmin_{n} \leq \sum a_{f,n} \cdot x_{f} \leq Nmax_{n}, \forall n \in N$$

2. Limit the volume of food consumed

$$\sum x_{f} \cdot V_{f} \leq Vmax$$

3. Consumption lower bound

$$x_{f} \geq 0$$

# Model

In [1]:
import pyomo.environ as pyo

In [2]:
model = pyo.ConcreteModel(name="Diet Problem", doc="Diet Problem")

In [3]:
FOODS = ["Cheeseburger", "Fries"]
NUTRIENTS = ["Calories", "Protein", "Fat"]

model.F =pyo.Set(initialize=FOODS, doc="Food Set")
model.N = pyo.Set(initialize=NUTRIENTS, doc="Nutrient Set")

In [4]:
cf = {"Cheeseburger": 1.84, 
    "Fries": 0.67}

afn = {
    ("Cheeseburger", "Calories"): 510,
    ("Cheeseburger", "Protein"): 34,
    ("Cheeseburger", "Fat"): 26,
    ("Fries", "Calories"): 220,
    ("Fries", "Protein"): 4,
    ("Fries", "Fat"): 19
}

Nminn = {
    "Calories": 2000,
    "Protein": 55,
    "Fat": 30
}

Nmaxn = {
    "Calories": 22000000,
    "Protein": 1000000,
    "Fat": 400000000
}

Vmax = 70

Vf = {
    "Cheeseburger": 5,
    "Fries": 5
}

model.cf = pyo.Param(model.F, initialize=cf, doc="Cost of Food")
model.afn = pyo.Param(model.F, model.N , initialize=afn, doc="Amount of Nutrients per Food")
model.Nminn = pyo.Param(model.N, initialize=Nminn, doc="Minimum Amount of Nutrients")
model.Nmaxn = pyo.Param(model.N, initialize=Nmaxn, doc="Maximum Amount of Nutrients")
model.Vmax = pyo.Param(initialize=Vmax, doc="Maximum Volume of Food")
model.Vf = pyo.Param(model.F, initialize=Vf, doc="Volume of Food per serving")

In [5]:
model.xf = pyo.Var(model.F, domain=pyo.NonNegativeReals, doc="Amount of serving food f")

In [6]:
#objective function

def cost_function(model):
    return sum(model.cf[f] * model.xf[f] for f in model.F)

model.objective = pyo.Objective(rule=cost_function, sense=pyo.minimize, doc="Total Cost of Food per serving")

In [7]:
#constraints
#1. Nutrient constraints
def nutrient_limit_max(model, n):
    return sum(model.afn[f,n] * model.xf[f] for f in model.F) <= model.Nmaxn[n]

def nutrient_limit_min(model, n):
    return model.Nminn[n] <= sum(model.afn[f,n] * model.xf[f] for f in model.F)

model.nutrient_limit_max = pyo.Constraint(model.N, rule=nutrient_limit_max, doc="Maximum Amount of Nutrients")
model.nutrient_limit_min = pyo.Constraint(model.N, rule=nutrient_limit_min, doc="Minimum Amount of Nutrients")


In [8]:
#2. Volume constraint
def volume_limit(model):
    return sum(model.Vf[f] * model.xf[f] for f in model.F) <= model.Vmax

model.volume_limit = pyo.Constraint(rule=volume_limit, doc="Maximum Volume of Food")

In [9]:
model.pprint()

Diet Problem

    3 Set Declarations
        F : Food Set
            Size=1, Index=None, Ordered=Insertion
            Key  : Dimen : Domain : Size : Members
            None :     1 :    Any :    2 : {'Cheeseburger', 'Fries'}
        N : Nutrient Set
            Size=1, Index=None, Ordered=Insertion
            Key  : Dimen : Domain : Size : Members
            None :     1 :    Any :    3 : {'Calories', 'Protein', 'Fat'}
        afn_index : Size=1, Index=None, Ordered=True
            Key  : Dimen : Domain : Size : Members
            None :     2 :    F*N :    6 : {('Cheeseburger', 'Calories'), ('Cheeseburger', 'Protein'), ('Cheeseburger', 'Fat'), ('Fries', 'Calories'), ('Fries', 'Protein'), ('Fries', 'Fat')}

    6 Param Declarations
        Nmaxn : Maximum Amount of Nutrients
            Size=3, Index=N, Domain=Any, Default=None, Mutable=False
            Key      : Value
            Calories :  22000000
                 Fat : 400000000
             Protein :   1000000
        Nm

## Solve the model

In [10]:
solver = pyo.SolverFactory('appsi_highs')

In [11]:
solver.solve(model, tee=True)

Presolving model
6 rows, 2 cols, 12 nonzeros
4 rows, 2 cols, 8 nonzeros
Presolve : Reductions: rows 4(-3); columns 2(-0); elements 8(-6)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(530) 0s
          2     6.3070772059e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  6.3070772059e+00
HiGHS run time      :          0.01


{'Problem': [{'Lower bound': 6.307077205882354, 'Upper bound': 6.307077205882354, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 1}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Termination message': 'TerminationCondition.optimal'}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [12]:
model.objective.expr()

6.307077205882354

In [13]:
model.xf.display()

xf : Amount of serving food f
    Size=2, Index=F
    Key          : Lower : Value              : Upper : Fixed : Stale : Domain
    Cheeseburger :     0 : 0.7536764705882355 :  None : False : False : NonNegativeReals
           Fries :     0 :            7.34375 :  None : False : False : NonNegativeReals


In [14]:
model.nutrient_limit_min.display()

nutrient_limit_min : Size=3
    Key      : Lower  : Body               : Upper
    Calories : 2000.0 :             2000.0 :  None
         Fat :   30.0 : 159.12683823529412 :  None
     Protein :   55.0 :  55.00000000000001 :  None


In [15]:
model.nutrient_limit_min["Calories"].pprint()

{Member of nutrient_limit_min} : Minimum Amount of Nutrients
    Size=3, Index=N, Active=True
    Key      : Lower  : Body                                 : Upper : Active
    Calories : 2000.0 : 510*xf[Cheeseburger] + 220*xf[Fries] :  +Inf :   True


In [16]:
model.afn.display()

afn : Amount of Nutrients per Food
    Size=6, Index=afn_index, Domain=Any, Default=None, Mutable=False
    Key                          : Value
    ('Cheeseburger', 'Calories') :   510
         ('Cheeseburger', 'Fat') :    26
     ('Cheeseburger', 'Protein') :    34
           ('Fries', 'Calories') :   220
                ('Fries', 'Fat') :    19
            ('Fries', 'Protein') :     4
